**In this part on Spark we used a subset filtered by Left and Right and English Language**

# Network Construction from Russian Trolls Dataset

This code performs the following steps to construct a retweet network from the `russian_trolls_spark` table:

1. **Load the DataFrame**  
   The dataset is loaded from the Spark table `default.russian_trolls_spark` into a DataFrame called `df`.

2. **Create Vertices (Nodes)**  
   Each unique author in the dataset is represented as a node. The code selects the `author` column (renamed to `id`) and the `account_category`, then removes duplicates to ensure each node is unique.

3. **Create Edges (Retweets)**  
   Edges are created for each retweet event, representing a directed connection from the retweeter (`author`, as `src`) to the original author (`original_author`, as `dst`). Only rows where `retweet` is `True` and `original_author` is not null are included.

4. **Weight the Edges**  
   The number of times each author retweets another is counted to assign a weight to each edge. This is done by grouping by `src` and `dst` and counting the occurrences.

5. **Write Output Files**  
   The resulting vertices and weighted edges DataFrames are each written as single CSV files to `/FileStore/ira_nodes_single` and `/FileStore/ira_edges_single`, respectively, for further analysis or visualization.

In [0]:
from pyspark.sql.functions import col, count

# 1. Load the dataframe 
df = spark.table("default.russian_trolls_spark") 

# 2. Create Vertices (Nodes)
vertices = (df
    .select(
        col("author").alias("id"),
        "account_category"
    )
    .dropDuplicates()
)

# 3. Create Edges (Retweets)
edges_retweet = (df
    .filter(col("retweet") == True)
   .select(
        col("author").alias("src"),
        col("original_author").alias("dst")
    )
    .filter(col("dst").isNotNull())
)

# 4. Weight the Edges
edges_retweet_weighted = (edges_retweet
    .groupBy("src", "dst")
    .agg(count("*").alias("weight"))
)


print("Writing Single Node File...")
vertices.coalesce(1).write.mode("overwrite").csv("/FileStore/ira_nodes_single", header=True)

print("Writing Single Edge File...")
edges_retweet_weighted.coalesce(1).write.mode("overwrite").csv("/FileStore/ira_edges_single", header=True)

Writing Single Node File...
Writing Single Edge File...



This code builds a retweet network from a dataset of tweets using PySpark:

- **Vertices (Nodes):** Each unique author in the dataset becomes a node. The code selects the `author` and their `account_category`, removes duplicates, and creates a DataFrame where each row represents a unique user.

- **Edges:** An edge is created for every retweet, representing a directed connection from the user who retweeted (`author`, as `src`) to the original author of the tweet (`original_author`, as `dst`). Only rows where a retweet actually happened and the original author is known are included.

- **Edge Weights:** The code then counts how many times each user retweeted another. This count becomes the weight of the edge between two users, showing the strength or frequency of their retweet relationship.

This section of code builds a retweet network from a dataset of tweets using PySpark:

- **Vertices (Nodes):** Each unique author in the dataset becomes a node. The code selects the `author` and their `account_category`, removes duplicates, and creates a DataFrame where each row represents a unique user.

- **Edges:** An edge is created for every retweet, representing a directed connection from the user who retweeted (`author`, as `src`) to the original author of the tweet (`original_author`, as `dst`). Only rows where a retweet actually happened and the original author is known are included.

- **Edge Weights:** The code then counts how many times each user retweeted another. This count becomes the weight of the edge between two users, showing the strength or frequency of their retweet relationship.

In [0]:
from pyspark.sql.functions import col, count

# VERTICES: each author is a node
vertices = (
    df
    .select(
        col("author").alias("id"),
        "account_category"
    )
    .dropDuplicates()
)

# EDGES: a retweet creates a link from the retweeter → original_author
edges_retweet = (
    df
    .filter(col("retweet") == True)
    .select(
        col("author").alias("src"),
        col("original_author").alias("dst")
    )
    .filter(col("dst").isNotNull())
)

# EDGE WEIGHTS: number of times author A retweets author B
edges_retweet_weighted = (
    edges_retweet
    .groupBy("src", "dst")
    .agg(count("*").alias("weight"))
)

In [0]:
import pandas as pd
import json
import networkx as nx
from pyspark.sql.functions import col

# Nodes: author + account_category
nodes_pd = (
    vertices
    .select("id", "account_category")
    .dropDuplicates()
    .toPandas()
)

# Edges: src, dst, weight
edges_pd = (
    edges_retweet_weighted
    .select(
        col("src").alias("Source"),
        col("dst").alias("Target"),
        "weight"
    )
    .toPandas()
)


# Build NetworkX graph and extract Largest Connected Component


# Build undirected graph for connectivity (you can use DiGraph if you prefer direction)
G = nx.from_pandas_edgelist(
    edges_pd,
    source="Source",
    target="Target",
    edge_attr="weight"
)

# Largest Connected Component (LCC)
largest_cc = max(nx.connected_components(G), key=len)
subgraph = G.subgraph(largest_cc).copy()


# Build Pandas tables for nodes/edges in the LCC

nodes_pd_sub = pd.DataFrame({"id": list(subgraph.nodes())})

nodes_pd_sub = nodes_pd_sub.merge(
    nodes_pd[["id", "account_category"]],
    on="id",
    how="left"
)

edges_pd_sub = pd.DataFrame(
    [
        {"Source": u, "Target": v, "weight": d.get("weight", 1)}
        for u, v, d in subgraph.edges(data=True)
    ]
)


# Build JSON for D3.js


nodes = [
    {
        "id": str(row["id"]),
        "account_category": str(row["account_category"]),
    }
    for _, row in nodes_pd_sub.iterrows()
]

links = [
    {
        "source": str(row["Source"]),
        "target": str(row["Target"]),
        "weight": float(row["weight"]),
    }
    for _, row in edges_pd_sub.iterrows()
]

graph = {"nodes": nodes, "links": links}
graph_json = json.dumps(graph)


# HTML + D3 force-directed graph (Databricks displayHTML)


html = f"""
<div id="viz" style="width:100%; height:800px; border:1px solid #ccc;"></div>

<!-- Graph data as JSON -->
<script id="graph-data" type="application/json">
{graph_json}
</script>

<!-- D3.js -->
<script src="https://d3js.org/d3.v7.min.js"></script>

<style>
  #viz svg {{
    background-color: #f9f9f9;
  }}
  .tooltip {{
    position: absolute;
    pointer-events: none;
    padding: 6px 10px;
    background: white;
    border: 1px solid #aaa;
    border-radius: 4px;
    font-size: 12px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.2);
  }}
</style>

<script type="text/javascript">
  // Show JS errors directly inside the notebook
  window.onerror = function(message, source, lineno, colno, error) {{
    var v = document.getElementById("viz");
    v.innerHTML = "<pre style='color:red; white-space:pre-wrap;'>" +
                  "JS error: " + message + "\\n" +
                  "Source: " + source + ":" + lineno + ":" + colno +
                  "</pre>";
    return false;
  }};

  if (typeof d3 === "undefined") {{
    document.getElementById("viz").innerHTML =
      "<p style='color:red;'>d3 is not defined (failed to load https://d3js.org/d3.v7.min.js)</p>";
  }} else {{
    const width = document.getElementById("viz").clientWidth;
    const height = document.getElementById("viz").clientHeight;

    const svgRoot = d3.select("#viz").append("svg")
        .attr("width", width)
        .attr("height", height);

    const svg = svgRoot.append("g");

    svgRoot.call(
      d3.zoom()
        .scaleExtent([0.3, 5])
        .on("zoom", (event) => svg.attr("transform", event.transform))
    );

    const tooltip = d3.select("body").append("div")
        .attr("class", "tooltip")
        .style("opacity", 0);

    // Read graph data from JSON script tag
    const graph = JSON.parse(
      document.getElementById("graph-data").textContent
    );

    // Color by account_category
    const color = d3.scaleOrdinal(d3.schemeCategory10);

    // Edge width scale based on retweet weight
    const maxWeight = d3.max(graph.links, d => d.weight) || 1;
    const edgeWidth = d3.scaleLinear()
        .domain([1, maxWeight])
        .range([0.5, 4]);

    const simulation = d3.forceSimulation(graph.nodes)
        .force("link", d3.forceLink(graph.links).id(d => d.id).distance(80))
        .force("charge", d3.forceManyBody().strength(-150))
        .force("center", d3.forceCenter(width / 2, height / 2));

    const link = svg.append("g")
        .attr("stroke", "#aaa")
        .attr("stroke-opacity", 0.4)
      .selectAll("line")
      .data(graph.links)
      .enter().append("line")
        .attr("stroke-width", d => edgeWidth(d.weight));

    const node = svg.append("g")
      .selectAll("circle")
      .data(graph.nodes)
      .enter().append("circle")
        .attr("r", 7)
        .attr("fill", d => color(d.account_category))
        .attr("stroke", "#fff")
        .attr("stroke-width", 1.2)
        .on("mouseover", function(event, d) {{
          tooltip.transition().duration(200).style("opacity", 0.9);
          tooltip
            .html(
              "<b>" + d.id + "</b><br>" +
              "Category: " + d.account_category
            )
            .style("left", (event.pageX + 8) + "px")
            .style("top", (event.pageY - 28) + "px");

          link.style("stroke-opacity", function(l) {{
            return (l.source.id === d.id || l.target.id === d.id) ? 0.9 : 0.1;
          }});
          node.style("opacity", function(n) {{
            return (n.id === d.id) ? 1 : 0.3;
          }});
        }})
        .on("mouseout", function() {{
          tooltip.transition().duration(200).style("opacity", 0);
          link.style("stroke-opacity", 0.4);
          node.style("opacity", 1);
        }})
        .call(d3.drag()
          .on("start", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0.3).restart();
            d.fx = d.x;
            d.fy = d.y;
          }})
          .on("drag", function(event, d) {{
            d.fx = event.x;
            d.fy = event.y;
          }})
          .on("end", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0);
            d.fx = null;
            d.fy = null;
          }}));

    simulation.on("tick", function() {{
      link
        .attr("x1", d => d.source.x)
        .attr("y1", d => d.source.y)
        .attr("x2", d => d.target.x)
        .attr("y2", d => d.target.y);
      node
        .attr("cx", d => d.x)
        .attr("cy", d => d.y);
    }});
  }}
</script>
"""

displayHTML(html)


<!-- Graph data as JSON -->


<!-- D3.js -->

We continued the analysis in Gephi, where we calculated modularity using the Louvain algorithm. We then brought the results back into Spark for further analysis. From this point onward, we work with the new dataset that includes the modularity class assignments for each node.

In [0]:
#Loading Dataset with Modularity from Gephi 

nodes_comm = spark.read.csv("dbfs:/FileStore/shared_uploads/alessia.tonicello@studenti.luiss.it/ida_modularity-1.csv", header=True, inferSchema=True)

In [0]:

df = (
    spark.read.csv(
        "dbfs:/FileStore/shared_uploads/alessia.tonicello@studenti.luiss.it/ida_modularity-1.csv",
        header=True,
        sep=";",
        inferSchema=True
    )
)

from pyspark.sql.functions import col
import re

def normalize(name):
    name = name.strip().lower()
    name = re.sub(r'[^a-z0-9_]', '_', name)
    name = re.sub(r'_+', '_', name)
    return name.strip("_")

df = df.toDF(*[normalize(c) for c in df.columns])

df.printSchema()


root
 |-- id: string (nullable = true)
 |-- account_category: string (nullable = true)
 |-- modularity_class: integer (nullable = true)



# Distribution Per Community

This code creates a DataFrame, called nodes_comm, with three columns: id, account_category, and modularity_class. 
It then groups the nodes by modularity_class, counts the number of nodes in each community, and orders the communities from largest to smallest.


In [0]:
from pyspark.sql.functions import split

# Split the single column into three separate columns
nodes_comm_split = nodes_comm.withColumn(
    "id", split(nodes_comm.columns[0], ";")[0]
).withColumn(
    "account_category", split(nodes_comm.columns[0], ";")[1]
).withColumn(
    "modularity_class", split(nodes_comm.columns[0], ";")[2]
)

# Now group by modularity_class
community_dist = (
    nodes_comm_split
    .groupBy("modularity_class")
    .count()
    .orderBy("count", ascending=False)
)
display(community_dist)

modularity_class,count
121,44
129,39
64,33
49,25
35,25
31,22
69,18
39,15
118,10
59,8


Databricks visualization. Run in Databricks to view.

The distribution is consistent with what large social networks typically produce after running a community-detection algorithm such as Louvain.

Interpretation:
- **A few modularity classes with high counts**
These represent the large, dense communities. In a social network, these might be major interest groups, regional clusters, or highly interconnected subcultures. It’s normal for only a handful of classes to dominate the node count.

- **A long tail of many modularity classes with very small counts**
Most detected communities are tiny—often just a handful of nodes. These correspond to fringe groups, random small clusters, or nodes loosely connected to the main graph.

- **The plot shape (tall spike on the left, long flat tail on the right)**
This is exactly the expected heavy-tailed community size distribution found in real-world social graphs.

**Inspecting the Composition of the Largest Communities**

To better understand the structure of the network, we first identified which communities (modularity classes) contain more than 20 nodes. These larger clusters are more likely to represent meaningful coordination patterns rather than isolated interactions.

After selecting these communities, we joined them with the node dataset to examine the distribution of account categories within each group (e.g., LeftTroll, RightTroll, Real_User).
This allows me to see whether certain communities are dominated by specific types of accounts.

We then inspected one of the largest communities (modularity class 121) by listing all the nodes that belong to it.

In [0]:
display(
    nodes_comm_split
    .filter(col("modularity_class") == 121)
)

id;account_category;modularity_class,id,account_category,modularity_class
____PantheR;Real_User;121,____PantheR,Real_User,121
Br3wn;Real_User;121,Br3wn,Real_User,121
CaliAtl11;Real_User;121,CaliAtl11,Real_User,121
CatwalkDarling;Real_User;121,CatwalkDarling,Real_User,121
CourtneyCummz;Real_User;121,CourtneyCummz,Real_User,121
Crislex;Real_User;121,Crislex,Real_User,121
dollie37;Real_User;121,dollie37,Real_User,121
Drea_TheCreole;Real_User;121,Drea_TheCreole,Real_User,121
flgal83;Real_User;121,flgal83,Real_User,121
HIPHOPSISTERS;Real_User;121,HIPHOPSISTERS,Real_User,121


We select now the top communities with more than 20 nodes to investigate how they are structured

In [0]:
from graphframes import GraphFrame

# 1. Select only the communities with more than 20 nodes
big_comms = community_dist.filter(col("count") > 20).select("modularity_class")

# 2. Join with the nodes table to retrieve the account categories
big_comms_details = (
    nodes_comm_split
        .join(big_comms, on="modularity_class", how="inner")
        .groupBy("modularity_class", "account_category")
        .count()
        .orderBy("modularity_class", "account_category")
)

display(big_comms_details)

# 3. Prepare vertices and edges for the graph
vertices_big = nodes_comm_split.join(big_comms, on="modularity_class", how="inner").select("id", "account_category", "modularity_class")

# Load edges DataFrame if not already loaded
edges = spark.read.csv("dbfs:/FileStore/shared_uploads/alessia.tonicello@studenti.luiss.it/edges.csv", header=True, inferSchema=True)

edges_big = (
    edges
    .join(vertices_big, edges["Source"] == vertices_big["id"], "inner")
    .join(vertices_big.withColumnRenamed("id", "id_dst"), edges["Target"] == col("id_dst"), "inner")
    .select(
        edges["Source"].alias("src"),
        edges["Target"].alias("dst")
    )
)

# 4. Create the GraphFrame
g = GraphFrame(vertices_big.withColumnRenamed("id", "id"), edges_big)

modularity_class,account_category,count
121,LeftTroll,4
121,Real_User,40
129,LeftTroll,1
129,Real_User,38
31,LeftTroll,2
31,Real_User,20
35,LeftTroll,1
35,Real_User,24
49,LeftTroll,1
49,Real_User,24


Databricks visualization. Run in Databricks to view.

**Graph Interpretation**

These results show the internal composition of the largest communities detected by the modularity algorithm.
Community 121, which is one of the most populated clusters, includes a mix of accounts but appears to be driven primarily by specific account categories (as shown in the grouped summary).

By comparing the category distribution across communities, we can identify which groups are mainly formed by trolls (left/right) and which instead consist mostly of real users.
This helps clarify whether the detected communities reflect coordinated in-group behavior, cross-group interactions, or clusters created by organic user activity.

In [0]:
import json
import pandas as pd

# Convert Spark DataFrames to Pandas 

nodes_pd = (
    nodes_comm_split
    .select("id", "account_category", "modularity_class")
    .distinct()
    .toPandas()
)

edges_pd = edges.select("Source", "Target").toPandas()

# Keep only the 20 most connected nodes (highest degree)

# Compute degree = number of times an id appears as Source or Target
all_endpoints = pd.concat([edges_pd["Source"], edges_pd["Target"]], ignore_index=True)
top_ids = list(all_endpoints.value_counts().head(20).index)

# Filter nodes: only keep top 20 ids
nodes_pd_sub = nodes_pd[nodes_pd["id"].astype(str).isin(top_ids)].copy()

# Filter edges: only keep edges where both ends are in the top 20 ids
edges_pd_sub = edges_pd[
    edges_pd["Source"].astype(str).isin(top_ids)
    & edges_pd["Target"].astype(str).isin(top_ids)
].copy()

# Build Python structures for D3

nodes = [
    {
        "id": str(row["id"]),
        "account_category": str(row["account_category"]),
        "modularity_class": str(row["modularity_class"])
    }
    for _, row in nodes_pd_sub.iterrows()
]

links = [
    {
        "source": str(row["Source"]),
        "target": str(row["Target"])
    }
    for _, row in edges_pd_sub.iterrows()
]

graph = {"nodes": nodes, "links": links}
graph_json = json.dumps(graph)

# HTML + JS for D3 force layout (with error catcher) 

html = f"""
<div id="viz" style="width:100%; height:800px; border:1px solid #ccc;"></div>

<!-- Graph data as JSON -->
<script id="graph-data" type="application/json">
{graph_json}
</script>

<!-- D3.js -->
<script src="https://d3js.org/d3.v7.min.js"></script>

<style>
  #viz svg {{
    background-color: #f9f9f9;
  }}
  .tooltip {{
    position: absolute;
    pointer-events: none;
    padding: 6px 10px;
    background: white;
    border: 1px solid #aaa;
    border-radius: 4px;
    font-size: 12px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.2);
  }}
</style>

<script type="text/javascript">
  // Show JS errors directly inside the notebook
  window.onerror = function(message, source, lineno, colno, error) {{
    var v = document.getElementById("viz");
    v.innerHTML = "<pre style='color:red; white-space:pre-wrap;'>" +
                  "JS error: " + message + "\\n" +
                  "Source: " + source + ":" + lineno + ":" + colno +
                  "</pre>";
    return false;
  }};

  if (typeof d3 === "undefined") {{
    document.getElementById("viz").innerHTML =
      "<p style='color:red;'>d3 is not defined (failed to load https://d3js.org/d3.v7.min.js)</p>";
  }} else {{
    const width = document.getElementById("viz").clientWidth;
    const height = document.getElementById("viz").clientHeight;

    const svgRoot = d3.select("#viz").append("svg")
        .attr("width", width)
        .attr("height", height);

    const svg = svgRoot.append("g");

    svgRoot.call(
      d3.zoom()
        .scaleExtent([0.3, 5])
        .on("zoom", (event) => svg.attr("transform", event.transform))
    );

    const tooltip = d3.select("body").append("div")
        .attr("class", "tooltip")
        .style("opacity", 0);

    // Read graph data from the JSON script tag
    const graph = JSON.parse(
      document.getElementById("graph-data").textContent
    );

    const color = d3.scaleOrdinal(d3.schemeCategory10);

    const simulation = d3.forceSimulation(graph.nodes)
        .force("link", d3.forceLink(graph.links).id(d => d.id).distance(80))
        .force("charge", d3.forceManyBody().strength(-150))
        .force("center", d3.forceCenter(width / 2, height / 2));

    const link = svg.append("g")
        .attr("stroke", "#aaa")
        .attr("stroke-opacity", 0.4)
      .selectAll("line")
      .data(graph.links)
      .enter().append("line")
        .attr("stroke-width", 1.5);

    const node = svg.append("g")
      .selectAll("circle")
      .data(graph.nodes)
      .enter().append("circle")
        .attr("r", 8)
        .attr("fill", d => color(d.modularity_class))
        .attr("stroke", "#fff")
        .attr("stroke-width", 1.5)
        .on("mouseover", function(event, d) {{
          tooltip.transition().duration(200).style("opacity", 0.9);
          tooltip
            .html(
              "<b>" + d.id + "</b><br>" +
              "Category: " + d.account_category + "<br>" +
              "Community: " + d.modularity_class
            )
            .style("left", (event.pageX + 8) + "px")
            .style("top", (event.pageY - 28) + "px");

          link.style("stroke-opacity", function(l) {{
            return (l.source.id === d.id || l.target.id === d.id) ? 0.9 : 0.1;
          }});
          node.style("opacity", function(n) {{
            return (n.id === d.id) ? 1 : 0.3;
          }});
        }})
        .on("mouseout", function() {{
          tooltip.transition().duration(200).style("opacity", 0);
          link.style("stroke-opacity", 0.4);
          node.style("opacity", 1);
        }})
        .call(d3.drag()
          .on("start", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0.3).restart();
            d.fx = d.x;
            d.fy = d.y;
          }})
          .on("drag", function(event, d) {{
            d.fx = event.x;
            d.fy = event.y;
          }})
          .on("end", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0);
            d.fx = null;
            d.fy = null;
          }}))
        .on("dblclick", function(event, d) {{
            d.fx = null;
            d.fy = null;
            simulation.alpha(0.5).restart();
        }});

    simulation.on("tick", function() {{
      link
        .attr("x1", function(d) {{ return d.source.x; }})
        .attr("y1", function(d) {{ return d.source.y; }})
        .attr("x2", function(d) {{ return d.target.x; }})
        .attr("y2", function(d) {{ return d.target.y; }});
      node
        .attr("cx", function(d) {{ return d.x; }})
        .attr("cy", function(d) {{ return d.y; }});
    }});
  }}
</script>
"""

displayHTML(html)


<!-- Graph data as JSON -->


<!-- D3.js -->

### Largest Connected Component (LCC)

In [0]:
import pandas as pd
import json
import networkx as nx

# Build nodes_pd and edges_pd from Spark 

nodes_pd = (
    nodes_comm_split
    .select("id", "account_category", "modularity_class")
    .distinct()
    .toPandas()
)

edges_pd = edges.select("Source", "Target").toPandas()

# Build NetworkX graph and extract largest connected component

G = nx.from_pandas_edgelist(edges_pd, source="Source", target="Target")

largest_cc = max(nx.connected_components(G), key=len)
subgraph = G.subgraph(largest_cc).copy()

# Convert back to pandas nodes/edges for that subgraph

nodes_pd_sub = pd.DataFrame({"id": list(subgraph.nodes())})

nodes_pd_sub = nodes_pd_sub.merge(
    nodes_pd[["id", "account_category", "modularity_class"]],
    on="id",
    how="left"
)

edges_pd_sub = pd.DataFrame(
    [{"Source": u, "Target": v} for u, v in subgraph.edges()]
)

# Build JSON for D3

nodes = [
    {
        "id": str(row["id"]),
        "account_category": str(row["account_category"]),
        "modularity_class": str(row["modularity_class"])
    }
    for _, row in nodes_pd_sub.iterrows()
]

links = [
    {
        "source": str(row["Source"]),
        "target": str(row["Target"])
    }
    for _, row in edges_pd_sub.iterrows()
]

graph = {"nodes": nodes, "links": links}
graph_json = json.dumps(graph)

# HTML + D3 force-directed visualization

html = f"""
<div id="viz" style="width:100%; height:800px; border:1px solid #ccc;"></div>

<!-- Graph data as JSON -->
<script id="graph-data" type="application/json">
{graph_json}
</script>

<!-- D3.js -->
<script src="https://d3js.org/d3.v7.min.js"></script>

<style>
  #viz svg {{
    background-color: #f9f9f9;
  }}
  .tooltip {{
    position: absolute;
    pointer-events: none;
    padding: 6px 10px;
    background: white;
    border: 1px solid #aaa;
    border-radius: 4px;
    font-size: 12px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.2);
  }}
</style>

<script type="text/javascript">
  // Show JS errors directly inside the notebook
  window.onerror = function(message, source, lineno, colno, error) {{
    var v = document.getElementById("viz");
    v.innerHTML = "<pre style='color:red; white-space:pre-wrap;'>" +
                  "JS error: " + message + "\\n" +
                  "Source: " + source + ":" + lineno + ":" + colno +
                  "</pre>";
    return false;
  }};

  if (typeof d3 === "undefined") {{
    document.getElementById("viz").innerHTML =
      "<p style='color:red;'>d3 is not defined (failed to load https://d3js.org/d3.v7.min.js)</p>";
  }} else {{
    const width = document.getElementById("viz").clientWidth;
    const height = document.getElementById("viz").clientHeight;

    const svgRoot = d3.select("#viz").append("svg")
        .attr("width", width)
        .attr("height", height);

    const svg = svgRoot.append("g");

    svgRoot.call(
      d3.zoom()
        .scaleExtent([0.3, 5])
        .on("zoom", (event) => svg.attr("transform", event.transform))
    );

    const tooltip = d3.select("body").append("div")
        .attr("class", "tooltip")
        .style("opacity", 0);

    // Read graph data from JSON script tag
    const graph = JSON.parse(
      document.getElementById("graph-data").textContent
    );

    const color = d3.scaleOrdinal(d3.schemeCategory10);

    const simulation = d3.forceSimulation(graph.nodes)
        .force("link", d3.forceLink(graph.links).id(d => d.id).distance(80))
        .force("charge", d3.forceManyBody().strength(-150))
        .force("center", d3.forceCenter(width / 2, height / 2));

    const link = svg.append("g")
        .attr("stroke", "#aaa")
        .attr("stroke-opacity", 0.4)
      .selectAll("line")
      .data(graph.links)
      .enter().append("line")
        .attr("stroke-width", 1.5);

    const node = svg.append("g")
      .selectAll("circle")
      .data(graph.nodes)
      .enter().append("circle")
        .attr("r", 8)
        .attr("fill", d => color(d.modularity_class))
        .attr("stroke", "#fff")
        .attr("stroke-width", 1.5)
        .on("mouseover", function(event, d) {{
          tooltip.transition().duration(200).style("opacity", 0.9);
          tooltip
            .html(
              "<b>" + d.id + "</b><br>" +
              "Category: " + d.account_category + "<br>" +
              "Community: " + d.modularity_class
            )
            .style("left", (event.pageX + 8) + "px")
            .style("top", (event.pageY - 28) + "px");

          link.style("stroke-opacity", function(l) {{
            return (l.source.id === d.id || l.target.id === d.id) ? 0.9 : 0.1;
          }});
          node.style("opacity", function(n) {{
            return (n.id === d.id) ? 1 : 0.3;
          }});
        }})
        .on("mouseout", function() {{
          tooltip.transition().duration(200).style("opacity", 0);
          link.style("stroke-opacity", 0.4);
          node.style("opacity", 1);
        }})
        .call(d3.drag()
          .on("start", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0.3).restart();
            d.fx = d.x;
            d.fy = d.y;
          }})
          .on("drag", function(event, d) {{
            d.fx = event.x;
            d.fy = event.y;
          }})
          .on("end", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0);
            d.fx = null;
            d.fy = null;
          }}))
        .on("dblclick", function(event, d) {{
            d.fx = null;
            d.fy = null;
            simulation.alpha(0.5).restart();
        }});

    simulation.on("tick", function() {{
      link
        .attr("x1", function(d) {{ return d.source.x; }})
        .attr("y1", function(d) {{ return d.source.y; }})
        .attr("x2", function(d) {{ return d.target.x; }})
        .attr("y2", function(d) {{ return d.target.y; }});
      node
        .attr("cx", function(d) {{ return d.x; }})
        .attr("cy", function(d) {{ return d.y; }});
    }});
  }}
</script>
"""

displayHTML(html)


<!-- Graph data as JSON -->


<!-- D3.js -->

### Interpretation of the Visualization: Largest Connected Component (LCC)
It appears clean, well-connected, and color-coded by community (modularity class).

We can see several structural patterns:
- The fan-shaped patterns indicate central hubs with many leaf nodes, a typical signature of retweet networks.
- The denser areas correspond to strong sub-communities.
- Colors represent modularity clusters, i.e., communities identified by the modularity algorithm.
- This type of visualization is commonly found in academic work on echo chambers, coordinated behavior, and retweet cascades.

**What Does This Graph Show?**
1. Multiple Distinct Communities
The color groups highlight different communities:
- Brown → a very dense, almost circular cluster; likely a strong retweet hub.
- Green → a more distributed cluster, possibly a coordinated group.
- Grey → a moderately large community with several internal sub-clusters.
- Orange, red, purple → smaller but clearly defined groups.

2. Communities Are Connected
- The graph is not fragmented, meaning:
- There are bridges that connect one community to another.
- Some users act as bridge nodes, which play a critical role in information flow and influence across clusters.

3. “Flower / Fan” Structures
These characteristic shapes indicate:
- A central node retweeted or referenced by many others.
- A typical pattern when a leader account produces content and the surrounding accounts amplify it.
- In bot or propaganda detection, such formations are important structures to inspect.

### Cross-Tabulation of Community Structure by Account Category

In [0]:
cross = nodes_comm_split.crosstab("modularity_class", "account_category").show()
display(cross)

+---------------------------------+---------+---------+----------+
|modularity_class_account_category|LeftTroll|Real_User|RightTroll|
+---------------------------------+---------+---------+----------+
|                              467|        0|        1|         0|
|                              296|        0|        1|         0|
|                              125|        0|        0|         1|
|                              451|        0|        1|         0|
|                               51|        1|        0|         0|
|                              591|        0|        1|         0|
|                              124|        0|        0|         1|
|                              447|        0|        1|         0|
|                                7|        0|        0|         1|
|                              574|        0|        1|         0|
|                              613|        0|        1|         0|
|                              475|        0|        1|       

This table shows how different communities (modularity classes) are made up of various account types—Left Trolls, Right Trolls, and Real Users.

Each row is a community, and the columns tell us how many accounts of each type are in that group.
- Most communities here have just one member, and almost all of them are Real Users.
- Occasionally, a community is made up of a single Left Troll or Right Troll, meaning these troll accounts tend to be isolated and don’t form groups.
- In this sample, there aren’t any mixed communities—each group contains only one type of account, and just one account in total.

This pattern suggests that these small communities are really just isolated or weakly connected accounts, not actual groups working together. The algorithm puts them in their own community because they don’t interact much with others.

Looking at this cross-tab helps us spot which communities are:
- made up of just one political group,
- mostly real users, or
- possibly dominated by trolls.

The most interesting communities are the bigger ones (with more than one member). We’ll look at those separately to see if there are real patterns of coordination in the network.

### Cross-Group Edge Interactions Between Right Trolls, Left Trolls, and Real Users 

In this section, we enrich the edge list by attaching information about the account category (Right Troll, Left Troll, Real User) and modularity class for both the source and target nodes. This allows us to analyze how different political or behavioral groups interact with each other through retweets.

The main goal is to identify cross-group connections, especially edges that link:

- Right Troll to Left Troll
- Left Troll to Right Troll
- Right Troll to Right Troll
- Left Troll to Left Troll

We focus on troll-to-troll interactions and exclude Real Users from the main table, since the aim is to highlight patterns that may indicate organized or strategic activity.

After grouping edges by source and destination category, the resulting table shows:

- How many times a troll account retweeted another troll account
- Whether the interactions are mostly internal (within the same political side)
- Whether cross-group interactions (Right to Left or Left to Right) occur

This breakdown helps us see whether troll behavior is:

- Clustered within political factions (for example, Right Trolls mostly retweeting other Right Trolls)
- Or whether there is cross-amplification between factions (which could suggest coordinated attempts to influence narratives across ideological lines)

Understanding these patterns provides insight into:

- Echo chambers within the troll ecosystem
- Potential bridges between ideological groups
- Communication asymmetries, such as one side amplifying more than the other

This analysis complements the community detection results by showing how information flows between political groups, not just within modularity-defined clusters.

In [0]:

edges = spark.read.csv("dbfs:/FileStore/shared_uploads/alessia.tonicello@studenti.luiss.it/edges.csv", header=True, inferSchema=True)

In [0]:
from pyspark.sql.functions import col

nodes_src = nodes_comm_split.select(
    col("id").alias("src"),
    col("account_category").alias("src_cat"),
    col("modularity_class").alias("src_mod")
)

nodes_dst = nodes_comm_split.select(
    col("id").alias("dst"),
    col("account_category").alias("dst_cat"),
    col("modularity_class").alias("dst_mod")
)

In [0]:
edges_enriched = (
    edges
    .join(nodes_src, edges["Source"] == nodes_src["src"], how="left")
    .join(nodes_dst, edges["Target"] == nodes_dst["dst"], how="left")
)
display(edges_enriched)

Source,Target,Weight,src,src_cat,src_mod,dst,dst_cat,dst_mod
JOHNIEVOGUE,NormPeterson6,1,JOHNIEVOGUE,LeftTroll,121,NormPeterson6,Real_User,121
JOHNIEBOYCOOKS,GeauxGabby,1,JOHNIEBOYCOOKS,LeftTroll,31,GeauxGabby,Real_User,31
JRRBRTT,tedcruz,1,JRRBRTT,RightTroll,96,tedcruz,Real_User,96
LIZAHASJUST,MoChroi1965,1,LIZAHASJUST,RightTroll,112,MoChroi1965,Real_User,112
WOKEFROMDAY1,rarebre3d,1,WOKEFROMDAY1,LeftTroll,31,rarebre3d,Real_User,31
MALLORYJARED,NasirLeigh,1,MALLORYJARED,LeftTroll,121,NasirLeigh,Real_User,121
LAGONEHOE,Echecrates,1,LAGONEHOE,LeftTroll,35,Echecrates,Real_User,35
TRAVISRESPEK,SnottieDrippen,1,TRAVISRESPEK,LeftTroll,39,SnottieDrippen,Real_User,39
LELANDGRAVES_,jbmesser53,1,LELANDGRAVES_,RightTroll,43,jbmesser53,Real_User,43
WILLISBONNERR,BunnnB,1,WILLISBONNERR,LeftTroll,49,BunnnB,Real_User,31


In [0]:
from pyspark.sql.functions import count

bridge = (edges_enriched
    .filter(~(col("src_cat") == "real user") & ~(col("dst_cat") == "real user"))
    .groupBy("src_cat", "dst_cat")
    .agg(count("*").alias("n_edges"))
    .orderBy("n_edges", ascending=False)
)

display(bridge)

src_cat,dst_cat,n_edges
LeftTroll,Real_User,232
RightTroll,Real_User,56


In [0]:
import pandas as pd
import json

# Convert Spark DataFrame to pandas
bridge_pd = bridge.toPandas()

# Build list of unique categories (nodes)
categories = sorted(set(bridge_pd["src_cat"]) | set(bridge_pd["dst_cat"]))

nodes = [
    {
        "id": cat,
        "label": cat
    }
    for cat in categories
]

# Build links with weights (n_edges)
links = [
    {
        "source": str(row["src_cat"]),
        "target": str(row["dst_cat"]),
        "weight": int(row["n_edges"])
    }
    for _, row in bridge_pd.iterrows()
]

graph = {"nodes": nodes, "links": links}
graph_json = json.dumps(graph)

# HTML + D3.js for visualization
html = f"""
<div id="bridge-viz" style="width:100%; height:600px; border:1px solid #ccc;"></div>

<script id="bridge-data" type="application/json">
{graph_json}
</script>

<script src="https://d3js.org/d3.v7.min.js"></script>

<style>
  #bridge-viz svg {{
    background-color: #f9f9f9;
  }}
  .tooltip-bridge {{
    position: absolute;
    pointer-events: none;
    padding: 6px 10px;
    background: white;
    border: 1px solid #aaa;
    border-radius: 4px;
    font-size: 12px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.2);
  }}
</style>

<script type="text/javascript">
  // Show JS errors directly in the notebook
  window.onerror = function(message, source, lineno, colno, error) {{
    var v = document.getElementById("bridge-viz");
    v.innerHTML = "<pre style='color:red; white-space:pre-wrap;'>" +
                  "JS error: " + message + "\\n" +
                  "Source: " + source + ":" + lineno + ":" + colno +
                  "</pre>";
    return false;
  }};

  if (typeof d3 === "undefined") {{
    document.getElementById("bridge-viz").innerHTML =
      "<p style='color:red;'>d3 is not defined (failed to load https://d3js.org/d3.v7.min.js)</p>";
  }} else {{
    const width = document.getElementById("bridge-viz").clientWidth;
    const height = document.getElementById("bridge-viz").clientHeight;

    const svgRoot = d3.select("#bridge-viz").append("svg")
        .attr("width", width)
        .attr("height", height);

    const svg = svgRoot.append("g");

    svgRoot.call(
      d3.zoom()
        .scaleExtent([0.5, 4])
        .on("zoom", (event) => svg.attr("transform", event.transform))
    );

    const tooltip = d3.select("body").append("div")
        .attr("class", "tooltip-bridge")
        .style("opacity", 0);

    // Read graph data
    const graph = JSON.parse(
      document.getElementById("bridge-data").textContent
    );

    // Custom color mapping (swapped real user and right troll)
    const customColor = d3.scaleOrdinal()
      .domain(["left troll", "right troll", "real user"])
      .range(["#2ecc40", "#7fdbff", "#ff4136"]); // green, light blue, red

    const maxWeight = d3.max(graph.links, d => d.weight) || 1;
    const edgeWidth = d3.scaleLinear()
        .domain([1, maxWeight])
        .range([1, 6]);

    const simulation = d3.forceSimulation(graph.nodes)
        .force("link", d3.forceLink(graph.links).id(d => d.id).distance(150))
        .force("charge", d3.forceManyBody().strength(-500))
        .force("center", d3.forceCenter(width / 2, height / 2));

    const link = svg.append("g")
        .attr("stroke", "#aaa")
        .attr("stroke-opacity", 0.5)
      .selectAll("line")
      .data(graph.links)
      .enter().append("line")
        .attr("stroke-width", d => edgeWidth(d.weight));

    const node = svg.append("g")
      .selectAll("circle")
      .data(graph.nodes)
      .enter().append("circle")
        .attr("r", 18)
        .attr("fill", d => customColor(d.id))
        .attr("stroke", "#fff")
        .attr("stroke-width", 1.5)
        .on("mouseover", function(event, d) {{
          tooltip.transition().duration(200).style("opacity", 0.9);
          tooltip
            .html("<b>" + d.label + "</b>")
            .style("left", (event.pageX + 8) + "px")
            .style("top", (event.pageY - 28) + "px");

          link.style("stroke-opacity", function(l) {{
            return (l.source.id === d.id || l.target.id === d.id) ? 0.9 : 0.1;
          }});
          node.style("opacity", function(n) {{
            return (n.id === d.id) ? 1 : 0.3;
          }});
        }})
        .on("mouseout", function() {{
          tooltip.transition().duration(200).style("opacity", 0);
          link.style("stroke-opacity", 0.5);
          node.style("opacity", 1);
        }})
        .call(d3.drag()
          .on("start", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0.3).restart();
            d.fx = d.x;
            d.fy = d.y;
          }})
          .on("drag", function(event, d) {{
            d.fx = event.x;
            d.fy = event.y;
          }})
          .on("end", function(event, d) {{
            if (!event.active) simulation.alphaTarget(0);
            d.fx = null;
            d.fy = null;
          }}));

    // Add labels on top of nodes
    const labels = svg.append("g")
      .selectAll("text")
      .data(graph.nodes)
      .enter().append("text")
        .attr("text-anchor", "middle")
        .attr("dy", 4)
        .style("font-size", "10px")
        .style("pointer-events", "none")
        .text(d => d.label);

    simulation.on("tick", function() {{
      link
        .attr("x1", d => d.source.x)
        .attr("y1", d => d.source.y)
        .attr("x2", d => d.target.x)
        .attr("y2", d => d.target.y);

      node
        .attr("cx", d => d.x)
        .attr("cy", d => d.y);

      labels
        .attr("x", d => d.x)
        .attr("y", d => d.y);
    }});
  }}
</script>
"""

displayHTML(html)

- Each node = an account_category (e.g. bot, news feed, troll, etc.).
- Each edge = number of edges between src_cat and dst_cat.
- Edge thickness = n_edges (how many interactions between the two categories).

**Interpretation of the Category-to-Category Bridge Graph**
This graph shows how different account categories interact with each other based on retweet activity:
- Real_User → LeftTroll is the strongest connection, indicated by the thickest edge. This means real users retweeted left-troll accounts more frequently.
- Real_User → RightTroll also appears, but with a weaker connection, suggesting fewer interactions.
- The absence of a direct LeftTroll ↔ RightTroll link suggests limited cross-interaction between troll groups.

Overall, the structure highlights asymmetric retweet patterns, where real users tend to amplify content from troll categories more than trolls amplify each other.
